In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
file_path = './countries.csv'
country_dataframe = pd.read_csv(file_path)

In [3]:
country_list = country_dataframe["country"].tolist()
city_list = country_dataframe["city"].tolist()
latitude_list = country_dataframe["latitude"].tolist()
longitude_list = country_dataframe["longitude"].tolist()

print(city_list)
print(country_list)

[' Shenzhen', ' Manila', ' San Francisco', ' Taipei', ' Panama City']
['China', 'Philippines', 'United States', 'Taiwan', 'Panama']


In [4]:
# Configurando o clinete da API do Open-Meteo com cache e tente novamente em caso de erro
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [5]:
climate_url = "https://archive-api.open-meteo.com/v1/archive"
climate_params = {
	"latitude": latitude_list,
	"longitude": longitude_list,
	"start_date": "2022-01-01",
	"end_date": "2024-03-20",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "precipitation_sum", "wind_speed_10m_max", "et0_fao_evapotranspiration"]
}

climate_responses = openmeteo.weather_api(climate_url, params=climate_params)

In [6]:
climate_dataframe = pd.DataFrame()

for i in range(len(latitude_list)):
    climate_response = climate_responses[i]
    """ print(f"Coordinates {climate_response.Latitude()}°N {climate_response.Longitude()}°E")
    print(f"Elevation {climate_response.Elevation()} m asl")
    print(f"Timezone {climate_response.Timezone()} {climate_response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {climate_response.UtcOffsetSeconds()} s") """

    # Process daily data. The order of variables needs to be the same as requested.
    daily = climate_response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(5).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(7).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(8).ValuesAsNumpy()

    daily_data = { "country": country_list[i], 
                "city": city_list[i], 
                "date": pd.date_range(
    	        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	        freq = pd.Timedelta(seconds = daily.Interval()),
    	        inclusive = "left"
    )}
    daily_data["latitude"] = latitude_list[i]
    daily_data["longitude"] = longitude_list[i]
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

    daily_climate_dataframe = pd.DataFrame(data = daily_data)
    frames = [climate_dataframe, daily_climate_dataframe]
    climate_dataframe = pd.concat(frames)

climate_csv = climate_dataframe.to_csv('./daily_climate.csv', index=False)
climate_dataframe

,country,city,date,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,precipitation_sum,wind_speed_10m_max,et0_fao_evapotranspiration
0,China,Shenzhen,2022-01-01 00:00:00+00:00,22.5415,114.0596,22.065500,12.065500,16.417583,22.559217,9.601063,16.377062,0.000000,12.218805,2.520732
1,China,Shenzhen,2022-01-02 00:00:00+00:00,22.5415,114.0596,21.865499,13.365500,16.986334,21.509037,11.301391,16.587885,0.000000,13.493999,2.723996
2,China,Shenzhen,2022-01-03 00:00:00+00:00,22.5415,114.0596,22.315500,14.415501,17.221750,22.719521,13.914251,17.497478,0.000000,10.308831,2.518255
3,China,Shenzhen,2022-01-04 00:00:00+00:00,22.5415,114.0596,22.615499,15.915501,18.611334,22.353077,15.462727,18.333288,0.300000,15.745627,2.833407
4,China,Shenzhen,2022-01-05 00:00:00+00:00,22.5415,114.0596,25.365499,14.915501,19.330082,27.544249,13.256737,20.259867,0.500000,15.124284,2.748522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Panama,Panama City,2024-03-16 00:00:00+00:00,8.9833,-79.5167,33.112499,26.062500,28.902084,40.312523,30.302158,33.070972,4.199999,21.046917,5.106321
806,Panama,Panama City,2024-03-17 00:00:00+00:00,8.9833,-79.5167,33.012497,25.912500,28.879166,39.528912,29.677071,33.040577,1.900000,17.654688,5.251953
807,Panama,Panama City,2024-03-18 00:00:00+00:00,8.9833,-79.5167,32.562500,25.962500,28.554167,36.873623,29.600391,32.070156,1.000000,22.077717,4.816419
808,Panama,Panama City,2024-03-19 00:00:00+00:00,8.9833,-79.5167,33.212498,25.962500,28.562500,38.006416,29.833591,32.303864,1.600000,23.710857,5.033191


In [7]:
air_quality_url = "https://air-quality-api.open-meteo.com/v1/air-quality"
air_quality_params = {
	"latitude": latitude_list,
	"longitude": longitude_list,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
	"start_date": "2022-08-04",
	"end_date": "2024-03-20",
	"domains": "cams_global"
}
air_quality_responses = openmeteo.weather_api(air_quality_url, params=air_quality_params)

In [8]:
air_quality_dataframe = pd.DataFrame()

for i in range(len(latitude_list)):
    air_quality_response = air_quality_responses[i]
    """ print(f"Coordinates {air_quality_response.Latitude()}°N {air_quality_response.Longitude()}°E")
    print(f"Elevation {air_quality_response.Elevation()} m asl")
    print(f"Timezone {air_quality_response.Timezone()} {air_quality_response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {air_quality_response.UtcOffsetSeconds()} s") """

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = air_quality_response.Hourly()
    hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
    hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
    hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
    hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
    hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
    hourly_ozone = hourly.Variables(5).ValuesAsNumpy()

    hourly_data = {
                   "date": pd.date_range(
    	            start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	            freq = pd.Timedelta(seconds = hourly.Interval()),
    	            inclusive = "left"
    )}
    hourly_data["latitude"] = latitude_list[i]
    hourly_data["longitude"] = longitude_list[i]
    hourly_data["pm10"] = hourly_pm10
    hourly_data["pm2_5"] = hourly_pm2_5
    hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
    hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
    hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
    hourly_data["ozone"] = hourly_ozone

    air_quality_hourly_dataframe = pd.DataFrame(data = hourly_data)
    frames = [air_quality_dataframe, air_quality_hourly_dataframe]
    air_quality_dataframe = pd.concat(frames)


In [9]:
air_quality_dataframe['date'] = pd.to_datetime(air_quality_dataframe['date'])

time = '00:00:00+00:00'
mask = (air_quality_dataframe['date'].dt.time == pd.Timestamp(time).time())
filtered_air_quality_df = air_quality_dataframe.loc[mask]

air_quality_csv = filtered_air_quality_df.to_csv('./daily_air_quality.csv', index=False)
filtered_air_quality_df

,date,latitude,longitude,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
0,2022-08-04 00:00:00+00:00,22.5415,114.0596,26.600000,18.299999,496.0,44.450001,46.150002,35.0
24,2022-08-05 00:00:00+00:00,22.5415,114.0596,25.000000,16.100000,352.0,21.000000,22.299999,49.0
48,2022-08-06 00:00:00+00:00,22.5415,114.0596,21.700001,13.900000,244.0,16.650000,29.650000,24.0
72,2022-08-07 00:00:00+00:00,22.5415,114.0596,27.000000,17.100000,364.0,33.549999,49.150002,31.0
96,2022-08-08 00:00:00+00:00,22.5415,114.0596,15.800000,9.700000,202.0,17.750000,25.500000,41.0
...,...,...,...,...,...,...,...,...,...
14160,2024-03-16 00:00:00+00:00,8.9833,-79.5167,16.200001,10.000000,252.0,10.600000,2.900000,48.0
14184,2024-03-17 00:00:00+00:00,8.9833,-79.5167,19.299999,11.800000,294.0,12.300000,3.500000,54.0
14208,2024-03-18 00:00:00+00:00,8.9833,-79.5167,20.900000,13.200000,289.0,11.900000,3.300000,51.0
14232,2024-03-19 00:00:00+00:00,8.9833,-79.5167,20.600000,12.900000,283.0,10.700000,3.400000,53.0


In [10]:
merged_df = pd.merge(climate_dataframe,filtered_air_quality_df, on=['date','latitude', 'longitude'], how='left')
merged_df.to_csv('./weather.csv', index=False)
merged_df

,country,city,date,latitude,longitude,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,precipitation_sum,wind_speed_10m_max,et0_fao_evapotranspiration,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
0,China,Shenzhen,2022-01-01 00:00:00+00:00,22.5415,114.0596,22.065500,12.065500,16.417583,22.559217,9.601063,16.377062,0.000000,12.218805,2.520732,NaN,NaN,NaN,NaN,NaN,NaN
1,China,Shenzhen,2022-01-02 00:00:00+00:00,22.5415,114.0596,21.865499,13.365500,16.986334,21.509037,11.301391,16.587885,0.000000,13.493999,2.723996,NaN,NaN,NaN,NaN,NaN,NaN
2,China,Shenzhen,2022-01-03 00:00:00+00:00,22.5415,114.0596,22.315500,14.415501,17.221750,22.719521,13.914251,17.497478,0.000000,10.308831,2.518255,NaN,NaN,NaN,NaN,NaN,NaN
3,China,Shenzhen,2022-01-04 00:00:00+00:00,22.5415,114.0596,22.615499,15.915501,18.611334,22.353077,15.462727,18.333288,0.300000,15.745627,2.833407,NaN,NaN,NaN,NaN,NaN,NaN
4,China,Shenzhen,2022-01-05 00:00:00+00:00,22.5415,114.0596,25.365499,14.915501,19.330082,27.544249,13.256737,20.259867,0.500000,15.124284,2.748522,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,Panama,Panama City,2024-03-16 00:00:00+00:00,8.9833,-79.5167,33.112499,26.062500,28.902084,40.312523,30.302158,33.070972,4.199999,21.046917,5.106321,16.200001,10.0,252.0,10.6,2.9,48.0
4046,Panama,Panama City,2024-03-17 00:00:00+00:00,8.9833,-79.5167,33.012497,25.912500,28.879166,39.528912,29.677071,33.040577,1.900000,17.654688,5.251953,19.299999,11.8,294.0,12.3,3.5,54.0
4047,Panama,Panama City,2024-03-18 00:00:00+00:00,8.9833,-79.5167,32.562500,25.962500,28.554167,36.873623,29.600391,32.070156,1.000000,22.077717,4.816419,20.900000,13.2,289.0,11.9,3.3,51.0
4048,Panama,Panama City,2024-03-19 00:00:00+00:00,8.9833,-79.5167,33.212498,25.962500,28.562500,38.006416,29.833591,32.303864,1.600000,23.710857,5.033191,20.600000,12.9,283.0,10.7,3.4,53.0
